# Hyperparameter Tuning (Deep Learning Model)
 - 딥러닝 모델의 하이퍼 파라미터 튜닝은 **직접 모델을 정의**해서 진행
 - Fashion MNIST 데이터를 분류하는 CNN 모델을 예시로 작성

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from sklearn.datasets import fetch_openml
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score

In [4]:
# Data Loading
mnist = fetch_openml('Fashion-MNIST')
print(mnist.data.shape, mnist.target.shape)

# Data Shaping
x_feat = mnist.data.to_numpy().reshape(-1, 28, 28) / 255
y_target = mnist.target.to_numpy().astype('int8').reshape(-1, 1)

# Split into train, validation, test data
x_train, x_test, y_train, y_test = train_test_split(x_feat, y_target, test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(70000, 784) (70000,)


((56000, 28, 28), (56000, 1), (14000, 28, 28), (14000, 1))

In [5]:
# 사용자 모델 정의 (CNN 모델)
def my_classifier(filters, kernel_size, padding, pool_size, optimizer):
    clf = Sequential()
    clf.add(Input(batch_shape=(None, x_train.shape[1], x_train.shape[2], 1)))
    clf.add(Conv2D(filters=filters, kernel_size=kernel_size, strides=1, padding=padding, activation='relu'))
    clf.add(MaxPool2D(pool_size=pool_size))
    clf.add(Flatten())
    clf.add(Dense(10, activation='softmax'))
    clf.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return clf

clf = KerasClassifier(build_fn=my_classifier)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if sys.path[0] == '':


In [6]:
# 하이퍼 파라미터 정의
params = {
    'filters' : [5, 15],
    'kernel_size' : [5, 7],
    'padding' : ['valid', 'same'],
    'pool_size' : [1, 2],
    'batch_size': [32, 64],
    'epochs': [10],
    'optimizer': ['adam']}

my_model = KerasClassifier(build_fn=my_classifier)
grid = GridSearchCV(estimator=my_model, param_grid=params, cv=3, verbose=3)
grid.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  # This is added back by InteractiveShellApp.init_path()


Fitting 3 folds for each of 32 candidates, totalling 96 fits
Epoch 1/10
1167/1167 [==============================] - 15s 3ms/step - loss: 0.5613 - accuracy: 0.8021
Epoch 2/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.4137 - accuracy: 0.8549
Epoch 3/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.3629 - accuracy: 0.8749
Epoch 4/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.3315 - accuracy: 0.8835
Epoch 5/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.3118 - accuracy: 0.8882
Epoch 6/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.2959 - accuracy: 0.8947
Epoch 7/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.2801 - accuracy: 0.9013
Epoch 8/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.2690 - accuracy: 0.9060
Epoch 9/10
1167/1167 [==============================] - 3s 3ms/step - loss: 0.2591 - accuracy: 0.9079
Epoch 10/10
584/584 

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f694190aa10>,
             param_grid={'batch_size': [32, 64], 'epochs': [10],
                         'filters': [5, 15], 'kernel_size': [5, 7],
                         'optimizer': ['adam'], 'padding': ['valid', 'same'],
                         'pool_size': [1, 2]},
             verbose=3)

In [7]:
# 평가가 가장 좋았던 모델의 하이퍼 파라미터와 정확도 확인
best_model = grid.best_estimator_
print("CV 최고 정확도 =", grid.best_score_)
print("\n최적 파라미터 =", grid.best_params_)
print("최적 모델로 측정한 테스트 데이터 정확도 = {:.4f}".format(best_model.score(x_test, y_test)))

CV 최고 정확도 = 0.9016071160634359

최적 파라미터 = {'batch_size': 32, 'epochs': 10, 'filters': 15, 'kernel_size': 5, 'optimizer': 'adam', 'padding': 'same', 'pool_size': 2}
438/438 [==============================] - 1s 2ms/step - loss: 0.2665 - accuracy: 0.9084
최적 모델로 측정한 테스트 데이터 정확도 = 0.9084
